# Imports

In [2]:
import folium  # map rendering library
from sklearn.cluster import KMeans  # import k-means from clustering stage
# Matplotlib and associated plotting modules
import matplotlib.colors as colors
import matplotlib.cm as cm
from bs4 import BeautifulSoup  # web scraping libraries and packages
# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize
import requests  # library to handle requests
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim
import json  # library to handle JSON files
import numpy as np  # library to handle data in a vectorized manner
import pandas as pd  # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('Libraries imported.')

Libraries imported.


# Scraping the Wikipedia page

In [30]:
# GET request
toronto_data = requests.get(
    'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(toronto_data, 'html.parser')
print('Data downloaded!')

Data downloaded!


# Wrangling and cleaning

In [31]:
postalCodes = []
boroughs = []
neighborhoods = []
# iterate each row of table and find postalcode, borough and neighborhood
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodes.append(cells[0].text.rstrip('\n'))
        boroughs.append(cells[1].text.rstrip('\n'))
        neighborhoods.append(cells[2].text.rstrip('\n'))
        

# Reading into dataframe with additional cleaning

In [39]:
df_toronto = pd.DataFrame({"PostalCode": postalCodes,
                           "Borough": boroughs,
                           "Neighborhood": neighborhoods})

# Filter borough and neighborhoods that are not asssigned
df_toronto = df_toronto[df_toronto.Borough != "Not assigned"].reset_index(drop=True)


def not_assigned_neighborhood(r):
    """
    Function to assign borough values for not assigned neighborhoods
    """
    if r['Neighborhood'] == "Not assigned":
        return r["Borough"]
    else:
        return r['Neighborhood']
        
# Assigning borough values to not assgned neighborhoods               
df_toronto["Neighborhood"] = df_toronto.apply(not_assigned_neighborhood,axis=1)


# Sanity check for the filter
assert len(df_toronto[df_toronto.Borough == "Not assigned"])==0
assert len(df_toronto[df_toronto.Neighborhood == "Not assigned"])==0

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

df_toronto.head()

The dataframe has 10 boroughs and 103 neighborhoods.


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [40]:
df_toronto.shape

(103, 3)